Load artifacts

In [1]:
import joblib, pandas as pd, numpy as np

models = {
    'Logistic Regression': joblib.load("models/logistic_regression_model.pkl"),
    'SVM': joblib.load("models/svm_model.pkl"),
    'KNN': joblib.load("models/knn_model.pkl"),
    'Random Forest': joblib.load("models/random_forest_model.pkl"),
}

pre = joblib.load("models/preprocessor.pkl")   # contains scaler (and cat_cols metadata if you stored it)
feature_names = joblib.load("models/feature_names.pkl")
scaler = pre['scaler']


Define inference helpers

In [2]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

def enforce_schema(df_row, feature_names):
    for c in feature_names:
        if c not in df_row.columns:
            df_row[c] = np.nan
    return df_row[feature_names]

def prepare_input(input_dict, feature_names):
    df_row = pd.DataFrame([input_dict])
    df_row = df_row.replace(['', ' ', 'NA', 'NaN', 'nan', '?'], np.nan)
    df_row = enforce_schema(df_row, feature_names)
    # Force numeric for known numeric fields
    for c in ['age','trestbps','chol','thalach','oldpeak','ca']:
        if c in df_row.columns:
            df_row[c] = pd.to_numeric(df_row[c], errors='coerce')
    X_imp = imputer.fit_transform(df_row.values)
    X_sc  = scaler.transform(X_imp)
    return X_sc

def predict_single(input_dict, model_name='SVM'):
    X_sc = prepare_input(input_dict, feature_names)
    model = models[model_name]
    pred = int(model.predict(X_sc))
    prob = float(model.predict_proba(X_sc)[0,1])
    risk = "Low Risk" if prob < 0.3 else ("Moderate Risk" if prob < 0.7 else "High Risk")
    return {'prediction': pred, 'probability_disease': prob, 'risk_level': risk}


Quick test

In [3]:
sample = {
    'age': 57, 'sex': 1, 'cp': 2, 'trestbps': 130, 'chol': 250,
    'fbs': 0, 'restecg': 1, 'thalach': 150, 'exang': 0,
    'oldpeak': 1.2, 'slope': 1, 'ca': 0, 'thal': 2
}
predict_single(sample, 'Random Forest')


c:\Users\aman0\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\aman0\AppData\Local\Temp\ipykernel_10332\666754160.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = int(model.predict(X_sc))


{'prediction': 1,
 'probability_disease': 0.5525401268701421,
 'risk_level': 'Moderate Risk'}